# Lab 9 - P4-DPDK Pipeline Multicore Processing

This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. Specifically, the experiment deploys a slice with two sites. Each site consists of one node: server1 in site1 and server2 in site2. The topology is shown in the figure below. The lab provides an introduction to DPDK, a software-based packet processing acceleration tool. It demonstrates how to build a topology using real hardware NICs.

<figure style="text-align: center;">
    <img src="./labs_files/lab8/figs/01_fabric_topology.png" width="400px">
</figure>

# Introduction

## Core affinity

<div style="text-align: justify;">
Core affinity allows binding a process or multiple processes to a specific CPU core, ensuring that they only execute on that core. When conducting performance testing on a host with many cores, it's beneficial to run multiple instances of a process, each on a different core. This approach helps maximize CPU utilization [<a href="#References">1</a>]. Figure 1 illustrates the difference between single-core and multicore processing. </div>  <be>

<figure style="text-align: center;">
  <img src="./labs_files/lab9/figs/intro_01.png" width="400" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Sigle-core vs Multicore processing [<a href="#References">2</a>].</figcaption>
</figure>

## Receive Side Scaling (RSS)

<div style="text-align: justify;">
When a NIC receives a packet, it typically goes through a single CPU core for processing. This can become a bottleneck, especially when the traffic rate is high, as the CPU core can become overwhelmed with processing incoming packets. To alleviate this bottleneck, the NIC can distribute the packets across multiple CPU cores using the Receive Side Scaling (RSS) technique. RSS uses a hashing algorithm to determine which receive queue should handle each incoming packet as shown in Figure 2. The hashing algorithm uses the packet header information, such as the 5-tuple (source and destination IP addresses, source and destination port numbers, and protocol), to generate a hash value. This hash value is then used to determine the appropriate receive queue for the packet, and subsequently, the CPU core where it will be processed. Note that all the packets belonging to the same flow will be assigned to the same CPU core. The NIC then forwards the packet to the pipeline running on the CPU core [<a href="#References">1</a>]. For performance-sensitive applications, DPDK uses CPU affinity within its software APIs by explicitly designating specific cores for handling packet input and output on NIC Rx/Tx queues [<a href="#References">3,4</a>]. </div>  <be>

<figure style="text-align: center;">
  <img src="./labs_files/lab9/figs/intro_02.png" width="400" style="display: block; margin: 0 auto;">
  <figcaption>Figure 2. Receive Side Scaling (RSS).</figcaption>
</figure>

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Import the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the experiment slice

The following creates two nodes with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "P4DPDK_lab9"

In [2]:
slice = fablib.new_slice(name="P4DPDK_lab9")

### Step 3.2: Define the sites
The code below requests two random sites from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 1 SmartNIC</li>
    <li> 8 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
sites= fablib.get_random_sites(count=2, filter_function=lambda x: x['nic_connectx_6_available'] > 1 and x['cores_available'] > 8 and x['ram_available'] > 8 and x['disk_available'] > 20)

print (f'The selected sites are {sites[0]} and {sites[1]}')

The selected sites are NCSA and SALT


### Step 3.3: Creating the nodes
The code below creates two nodes: server1 and server2. The servers (server1 and server2) use the following:
<ul>
    <li> 8 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1 and server2 will be created in site2

<img src="./labs_files/lab8/figs/03_creating_nodes.png" width="400px"><br>

In [4]:
server1 = slice.add_node(name="server1", 
                      site=sites[0], 
                      cores=8, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=sites[1], 
                      cores=8, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./labs_files/lab8/figs/04_adding_nics.png" width="400px"><br>

In [5]:
server1_iface = server1.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_ConnectX_6', name='nic2').get_interfaces()[0]

### Step 3.5: Connecting server1 and server2
Create a network between server1 and server2 connecting them together

<img src="./labs_files/lab8/figs/05_connecting_nodes_server1_server2.png" width="400px"><br>

In [6]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, server2_iface])

### Step 3.6: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [7]:
slice.submit();


Retry: 10, Time: 245 sec


ID,1cdf2833-f782-4376-8bbe-17304ee6ecd6
Name,P4DPDK_lab9_TTTT
Lease Expiration (UTC),2024-09-23 16:46:56 +0000
Lease Start (UTC),2024-09-22 16:46:56 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
a23e99bd-9d6f-4778-9273-fdf011dbd4e6,server1,8,8,100,default_ubuntu_20,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe79:7ea7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe79:7ea7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
73884114-d955-4cfa-a40d-ec75a55ebff2,server2,8,8,100,default_ubuntu_20,qcow2,salt-w2.fabric-testbed.net,SALT,ubuntu,2001:400:a100:3010:f816:3eff:fe6e:65d1,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe6e:65d1,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
433bcf49-8430-44f9-9aa0-cd1faa9253dc,net1,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
server1-nic1-p1,p1,server1,net1,100,config,,B8:CE:F6:37:75:1A,enp7s0,enp7s0,fe80::bace:f6ff:fe37:751a,1,HundredGigE0/0/0/0
server1-nic1-p2,p2,server1,None,100,config,,B8:CE:F6:37:75:1B,enp8s0,enp8s0,None,1,None
server2-nic2-p1,p1,server2,net1,100,config,,B8:CE:F6:5D:45:6E,enp7s0,enp7s0,fe80::bace:f6ff:fe5d:456e,6,HundredGigE0/0/0/4
server2-nic2-p2,p2,server2,None,100,config,,B8:CE:F6:5D:45:6F,enp8s0,enp8s0,None,6,None



Time to print interfaces 257 seconds


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the Mellanox drivers, DPDK library, the P4 compiler (p4c), and all needed dependencies.

## Step 4.1 Appending list of servers
All servers are appended to a list to execute commands in parallel

In [8]:
servers = []

servers.append(slice.get_node(name="server1"))     
servers.append(slice.get_node(name="server2"))

server1 = servers[0]
server2 = servers[1]

## Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [9]:
from ipaddress import ip_address, IPv6Address

threads = []

for server in servers:
    if type(ip_address(server.get_management_ip())) is IPv6Address:
        server.upload_file('scripts/nat64.sh', 'nat64.sh')
        threads.append(server.execute_thread(f'chmod +x nat64.sh && ./nat64.sh'))

for thread in threads:
    thread.result()

## Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [10]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        sudo apt-get update;
        sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools;
        sudo pip3 install meson ninja
    '''))

for thread in threads:
    thread.result()

## Step 4.4 Installing Mellanox drivers
Since ConnectX-6 NICs are used in this lab, it is essential to install the supporting drivers. The code below downloads and installs Mellanox drivers on all servers while enabling DPDK

In [11]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        wget https://content.mellanox.com/ofed/MLNX_OFED-23.07-0.5.0.0/MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz; 
        tar xvfz MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz; 
        cd MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64; 
        echo "y" | sudo ./mlnxofedinstall --upstream-libs --dpdk --basic --without-fw-update --enable-sriov --hypervisor
    '''))
    
for thread in threads:
    thread.result()

## Step 4.5 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers

In [12]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        git clone http://dpdk.org/git/dpdk; 
        cd dpdk;
        sudo meson build;
        cd build;
        sudo ninja;
        sudo ninja install; 
        sudo ldconfig
    '''))

for thread in threads:
    thread.result()

## Step 4.6 Installing Pktgen
In this lab, we will send from server1 packets at a high rate to server2 using a DPDK-based packet generation tool called pktgen [<a href="#References">5</a>]. The code below downloads and installs Pktgen-DPDK on all servers

In [13]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        sudo git clone https://github.com/pktgen/Pktgen-DPDK; 
        sudo sed -i \"s/deps += \\[dependency('numa', required: true)\\]/deps += \\[dependency('numa', required: false)\\]/\" /home/ubuntu/Pktgen-DPDK/app/meson.build;
        sudo apt-get install -y cmake libpcap-dev libbsd-dev;
        cd Pktgen-DPDK &&  sudo meson build && sudo ninja -C build && cd build/ && sudo meson install
    '''))

for thread in threads:
    thread.result()

## Step 4.7 Reboot
After installing the ConnectX Mellanox divers, it is essential to reboot the servers to complete the installation process or to ensure that updates are applied correctly

In [14]:
for server in servers:
    server.os_reboot()

## Step 4.8 Build pipeline library
The code below builds the DPDK pipeline library in server 2 (on which the pipeline will be running) to put all its functions into effect 

In [15]:
stdout, stderr = server2.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

## Step 4.9 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline. In this lab, p4c is built from a version where the architecture has been modified.

In [16]:
stdout, stderr = server2.execute('source /etc/lsb-release && echo "deb http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${DISTRIB_RELEASE}/ /" | sudo tee /etc/apt/sources.list.d/home:p4lang.list && curl -fsSL https://download.opensuse.org/repositories/home:p4lang/xUbuntu_${DISTRIB_RELEASE}/Release.key | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/home_p4lang.gpg > /dev/null && sudo apt-get update && sudo apt install -y p4lang-p4c', quiet = True)
stdout, stderr = server2.execute("cd /usr/share/p4c/p4include/dpdk/ && sudo sed -i '769s/\\(.\\{6\\}\\)/\\1out/' pna.p4", quiet=True)

# Step 5: Configuring Network
In this step, we will assign IPv4 addresses to the interfaces of the servers and hardcode the MAC addresses. We will also configure forwarding and routing.

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/lab8/figs/07_getting_interfaces.png" width="400px"><br>

In [17]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()+'np0'
print(f'server1_iface: {server1_iface_name}')

node2_iface = server2.get_interface(network_name='net1') 
server2_iface_name = node2_iface.get_device_name()+'np0'
print(f'server2_iface: {server2_iface_name}')

server1_iface: enp7s0np0
server2_iface: enp7s0np0


## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./labs_files/lab8/figs/08_interfaces_up.png" width="400px"><br>

In [18]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>server2_iface_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
</ul>

<img src="./labs_files/lab8/figs/09_mac_addresses.png" width="400px"><br>

In [19]:
server1_iface_MAC = '00:00:00:00:00:01'
server2_iface_MAC = '00:00:00:00:00:02'

## Step 5.4 Configuring the IP and MAC addresses on server1_iface and server2_iface

We will use the network 192.168.10.0/24 between server1 and server2. We will assign the IP address 192.168.10.1 to server1's interface and 192.168.10.2 to its neighboring interface on server2.

<img src="./labs_files/lab8/figs/10_IPs_10.png" width="400px"><br>

In [20]:
server1_server2_subnet = "192.168.10.0/24"
server1_ip = '192.168.10.1/24'
server2_ip = '192.168.10.2/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

## Step 5.5: Enable forwarding on server2

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the server2.

In [21]:
stdout, stderr = server2.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)

## Step 5.6: Configure ARP

In this step, we will configure static ARP entries on server1, and server2.

In [22]:
gw1 = server2_ip.split('/')[0]
gw2 = server1_ip.split('/')[0]
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {server2_iface_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {server1_iface_MAC}')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

## Step 5.7: Mellanox devices

In this step, we will inspect and start all Mellanox devices.

In [23]:
stdout, stderr = server1.execute(f'sudo ibdev2netdev')
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)
stdout, stderr = server1.execute(f'sudo mst start', quiet=True)
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)

stdout, stderr = server2.execute(f'sudo ibdev2netdev')
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)
stdout, stderr = server2.execute(f'sudo mst start', quiet=True)
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)

mlx5_0 port 1 ==> enp7s0np0 (Up)
 sudo: unable to resolve host server1: Name or service not known
 mlx5_1 port 1 ==> enp8s0np1 (Down)
mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)
 sudo: unable to resolve host server2: Name or service not known
 

In the output above, you can see that there are two interfaces that are turned down. This is because each server has a Connect-X6 NIC attached, which is a dual-port NIC, and only one port in each NIC is used in this topology. 

# Step 6: Compiling the P4 program

In this lab, we will not modify the P4 code in which we implement a simple packet reflector. Instead, we will just compile it using the p4c-dpdk compiler. Note that in this P4 code the Portable NIC Architecture (PNA) is used.

To upload and compile the P4 program, issue the following command.

In [24]:
server2.upload_file('labs_files/lab9/lab9.p4','lab9.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna lab9.p4 -o lab9.spec')
stdout, stderr = server2.execute(f'ls')

 sudo: unable to resolve host server2: Name or service not known
 MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64
MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz
Pktgen-DPDK
dpdk
lab9.p4
lab9.spec
nat64.sh


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab9.spec``` file which is a specification file needed to run the pipeline.

# Step 7: Implementing the P4-DPDK pipeline

## Step 7.1: Preparing the P4-DPDK CLI script
Each P4-DPDK pipeline is built through the CLI script. In this subsection, we will write the CLI script in which the pipeline is created and built.

Click on [lab9.cli](./labs_files/lab9/lab9.cli) to open the CLI file in the editor.

<img src="./labs_files/lab9/figs/7_02_01.png" width="650px"><br>
We can see that the lab9.cli file is empty and we have to fill it.

<hr>

We will start by generating the pipeline code and building the shared object. Write the following in the lab9.cli file.

    pipeline codegen /home/ubuntu/lab9.spec /tmp/lab9.c
    pipeline libbuild /tmp/lab9.c /tmp/lab9.so

<img src="./labs_files/lab9/figs/7_02_02.png" width="800"><br>

In the figure above the ```codegen``` function (line 2) is used to generate the C code of the compiled user application in the spec file. This function takes two arguments, the path to the specification file compiled ```/home/ubuntu/lab9.spec``` and the generated code is placed in a temporary directory ```/tmp/lab9.c```.

The ```libbuild``` function (line 3) is used to generate a shared object to execute the application. This function takes two arguments, the path to the C code ```/tmp/lab9.c``` and the generated shared object is placed in a temporary ```/tmp/lab9.so```.

<hr>

Now we will list the DPDK devices with customized parameters that match our setup. Write the following in the lab9.cli file.

    mempool MEMPOOL0 meta 0 pkt 9500 pool 128K cache 256 numa 0

<img src="./labs_files/lab9/figs/7_02_03.png" width="800"><br>

The line of code in the figure above (line 6) is used to create DPDK objects like a memory pool mempool with parameters to setup each DPDK device.  A memory pool MEMPOOL0 is defined as follows: 

•	```mempool```: create a memory pool object associated with a given name. <br>
•	```meta```: specifies the private size of the memory buffer in bytes, which is the memory allocated for an application’s private data.<br>
•	```pkt```: specifies the private size of the memory buffer in bytes, which is the memory allocated for an application to store data associated with a packet.<br>
•	```pool```: the size of the defined memory pool specified in bytes.<br>
•	```cache```: the cache size in bytes which should be a power of 2. <br>
•	```numa```: pinned NUMA node ID.


Execute the command below to inspect the name the interface on server2

In [34]:
stdout, stderr = server2.execute('lspci | grep ConnectX | awk \'{print "0000:" $1}\'| head -n 1')

0000:07:00.0


Now we will list the DPDK ethernet devices with customized parameters that match our setup. Write the following in the lab9.cli file.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
It is important that the interface name matches the output of the previous command as it should also be used in the I/O specification file. This is the notation considered by DPDK while the operating system assigns different tags to the interfaces. This is an example where the interface name is 0000:07:00.0
</div>

    ethdev 0000:07:00.0 rxq 4 1024 MEMPOOL0 txq 4 1024 promiscuous on rss 0 1 2 3

<img src="./labs_files/lab9/figs/7_02_04.png" width="800"><br>

The block of code in the figure above (line 9) is used to create an ethernet device ```ethdev``` with customized parameters.

Ethernet devices which are the interfaces linked to the pipeline are defined as follows:

•	```ethdev```: ethernet device name (the attached devices net_tap0 and net_tap1 are virtual ethernet devices with their instances created when the pipeline is invoked). <br>
•	```rxq```: receiving queue parameters (number of receiving queues, queue size (bytes), memory pool name).<br>
•	```txq```: transmitting queue parameters (number of transmitting queues, queue size (bytes)). <br>
•	```promiscuous```: a mode that allows a network device to read each network packet that arrives (on / off).<br>
•	```rss```: the queue IDs to which the packets are going to be directed.<br>

<hr>

Now we will list the P4-DPDK pipelines. Write the following in the lab9.cli file.

    pipeline PIPELINE0 build lib /tmp/lab9.so io /home/ubuntu/ethdev0.io numa 0
    pipeline PIPELINE1 build lib /tmp/lab9.so io /home/ubuntu/ethdev1.io numa 0
    pipeline PIPELINE2 build lib /tmp/lab9.so io /home/ubuntu/ethdev2.io numa 0
    pipeline PIPELINE3 build lib /tmp/lab9.so io /home/ubuntu/ethdev3.io numa 0

<img src="./labs_files/lab9/figs/7_02_05.png" width="800"><br>

In the figure above the ```build``` function (lines 12 - 16) is used to create a pipeline object, for example ```PIPELINE0```. This function takes the path of the shared object library ```lib  /tmp/lab9.so```, the path of the I/O file (which will be discussed in detail in the next subsection) ```io  /home/ubuntu/ethdev.io``` and the numa node ID ```numa  0```. Each pipeline is mapped to a different I/O file but they are all pinned to the same NUMA node.

<hr>

Now we will map the created pipeline to a CPU thread. Write the following in the lab9.cli file.

    pipeline PIPELINE0 enable thread 1
    pipeline PIPELINE1 enable thread 2
    pipeline PIPELINE2 enable thread 3
    pipeline PIPELINE3 enable thread 4

<img src="./labs_files/lab9/figs/7_02_06.png" width="800"><br>

In the figure above the ```enable thread``` function (lines 18 - 21) is used to map the pipeline to a CPU thread. For example ```PIPELINE0``` is pinned to the CPU thread with ID 1.

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 7.2: Preparing the I/O scripts
The stream of packets within a P4-DPDK must be configured. In this subsection, we will write the I/O scripts which are the configuration files for each pipeline input and output stream.

Execute the command below to inspect the name the interface on server2

In [24]:
stdout, stderr = server2.execute('lspci | grep ConnectX | awk \'{print "0000:" $1}\'| head -n 1')

0000:07:00.0


The output of the command above will be used to refer to the interface name in the I/O scripts

Click on [ethdev0.io](./labs_files/lab9/ethdev0.io) to open the I/O file in the editor.

<img src="./labs_files/lab9/figs/7_03_01.png" width="650px"><br>

We can see that the ethdev0.io file is empty and we have to fill it.

<hr>

We will start by defining the pipeline input port. Write the following in the ethdev0.io file.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
The step shows an example where the interface name is 0000:07:00.0. Modify the interface name to match the output of the command executed at the beginning of this section.
</div>

    port in 0 ethdev 0000:07:00.0 rxq 0 bsz 32

<img src="./labs_files/lab9/figs/7_03_02.png" width="650px"><br>

Input ports to the pipeline are defined as follows:

•	```port in```: the pipeline input port ID<br>
•	```ethdev```: the ethernet device associated with the defined port<br>
•	```rxq```: the receiving queue ID<br>
•	```bsz```: burst size (packets)<br>

In the figure above the ```port in``` function (line 2) is used to define the input port to the pipeline. In this case, 0 is assigned as the port ID of the port. ```0000:07:00.0``` is the name of ethernet device associated with ports 0. 

Every packet received at an input port is then forwarded to a receiving queue in the pipeline as determined by the ```rxq``` parameter which holds a value representing the receiving queue ID. The port will forward packets to a single queue with ID 0. The ```bsz``` parameter represents the burst size. DPDK attempts to aggregate the cost of processing each packet individually by processing packets in bursts, and the burst size is set to 32.

<hr>

Now we will define the pipeline output port. Write the following in the ethdev0.io file.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
The step shows an example where the interface name is 0000:07:00.0. Modify the interface name to match the output of the command executed at the beginning of this section.
</div>

    port out 0 ethdev 0000:07:00.0 txq 0 bsz 32

<img src="./labs_files/lab9/figs/7_03_03.png" width="650px"><br>

Output ports to the pipeline are defined as follows:

•	```port out```: the pipeline output port ID<br>
•	```ethdev```: the ethernet device associated with the defined port<br>
•	```txq```: the transmitting queue ID<br>
•	```bsz```: burst size (packets)<br>

In the figure above the ```port out``` function (lines 4) is used to define the output port from the pipeline. Similar to the ```port in``` function, when this function is called, it is followed by the port ID and ethernet device interface ID. Every packet delivered at an output port is then forwarded to a transmitting queue in the pipeline as determined by the ```txq``` parameter which holds a value representing the transmitting queue ID. The port will forward packets to a single queue with ID 0. The burst size ```bsz``` is set to 32.

<hr>

Save the changes by pressing ```Ctrl+s```.

<hr>

Click on [ethdev1.io](./labs_files/lab9/ethdev1.io) to open the I/O file in the editor.

We will define the input and output ports of the second pipeline. Write the following in the ethdev1.io file.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
The step shows an example where the interface name is 0000:07:00.0. Modify the interface name to match the output of the command executed at the beginning of this section.
</div>

    port in 0 ethdev 0000:07:00.0 rxq 1 bsz 32
    port out 0 ethdev 0000:07:00.0 rxq 1 bsz 32

<img src="./labs_files/lab9/figs/7_03_04.png" width="800px"><be>

<hr>

Save the changes by pressing ```Ctrl+s```.

<hr>

Click on [ethdev2.io](./labs_files/lab9/ethdev2.io) to open the I/O file in the editor.

We will define the input and output ports of the third pipeline. Write the following in the ethdev2.io file.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
The step shows an example where the interface name is 0000:07:00.0. Modify the interface name to match the output of the command executed at the beginning of this section.
</div>

    port in 0 ethdev 0000:07:00.0 rxq 2 bsz 32
    port out 0 ethdev 0000:07:00.0 rxq 2 bsz 32

<img src="./labs_files/lab9/figs/7_03_05.png" width="800px"><be>

<hr>

Save the changes by pressing ```Ctrl+s```.

<hr>

Click on [ethdev3.io](./labs_files/lab9/ethdev3.io) to open the I/O file in the editor.

We will define the input and output ports of the fourth pipeline. Write the following in the ethdev3.io file.

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
The step shows an example where the interface name is 0000:07:00.0. Modify the interface name to match the output of the command executed at the beginning of this section.
</div>

    port in 0 ethdev 0000:07:00.0 rxq 3 bsz 32
    port out 0 ethdev 0000:07:00.0 rxq 3 bsz 32

<img src="./labs_files/lab9/figs/7_03_06.png" width="800px"><be>

<hr>

Save the changes by pressing ```Ctrl+s```.

# Step 8: Running the P4-DPDK pipeline
Now that all the required scripts are prepared, we can run the pipeline.

## Step 8.1: Uploading files
The following code uploads the CLI and I/O scripts to server2.

In [25]:
server2.upload_file('labs_files/lab9/lab9.cli','lab9.cli')
server2.upload_file('labs_files/lab9/ethdev0.io','ethdev0.io')
server2.upload_file('labs_files/lab9/ethdev1.io','ethdev1.io')
server2.upload_file('labs_files/lab9/ethdev2.io','ethdev2.io')
server2.upload_file('labs_files/lab9/ethdev3.io','ethdev3.io')

<SFTPAttributes: [ size=86 uid=1000 gid=1000 mode=0o100664 atime=1727024928 mtime=1727024928 ]>

## Step 8.2: Reserving hugepages
Configure the number of hugepages in the system by typing the following command. 

In [26]:
threads = []

for server in servers:
    threads.append(server.execute_thread(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"'))

for thread in threads:
    thread.result()

Hugepage reservation is done by setting the number of hugepages required to the ```nr_hugepages``` file in the kernel corresponding to a specific page size (in Kilobytes).

The ```echo``` command is used to print a value which in this case is ```1024``` representing the number of hugepages. The ```>``` symbol is a redirection operator that redirects the output of the previous command (echo 1024) to the file specified in the following path: ```/sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages```

## Step 8.3: Opening a terminal

Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab9/figs/terminal1.gif" width="800px"><br>

Copy the output of the command below and paste it in the terminal to enter to server2.

In [27]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3010:f816:3eff:fe6e:65d1'

## Step 8.4: Running the pipeline

Run the following commands in the terminal:
    
    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/lab9.cli
    
<img src="./labs_files/lab9/figs/pipeline_run.png" width="700"><br>

In the figure above, the command is used to run the DPDK pipeline application considering the following arguments:

•	```examples/pipeline/build/pipeline```: the path to the executable DPDK pipeline application.<br>
•	```-c```: this parameter is used to specify the hexadecimal bitmask of the cores to run on. In this case, (0x1F) indicated that 4 cores are reserved for the pipelines and one extra core is needed for other processes.<br>
•	```-s```: this parameter is used to specify the path to the CLI script file to be run at application startup ```/home/ubuntu/lab9.cli```.<br>

Note that when the DPDK pipeline runs, the CLI script is printed in the terminal. If any problems are encountered while running the pipeline, error messages will be shown within the printed CLI script.


# Setp 9: Testing the application
To test the application, we will send from server1 packets at a high rate to server2 using a DPDK-based packet generation tool called pktgen. 

## Step 9.1: Opening a new terminal

Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab9/figs/terminal2.gif" width="800px"><br>

Copy the output of the command below and paste it in the terminal to enter to server1.

In [28]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe79:7ea7'

## Step 9.2: Running pktgen

Run the command below to know which interface is being used on server 1 to communicate with server 2 

In [49]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


The output of this command will be used to refer to the interface ID in the command invoked to run pktgen

Run the following commands in the terminal to run pktgen:

<div style="background-color: #e0f7fa; border: 1px solid #b2ebf2; padding: 10px; border-radius: 5px;">
It is important that the interface name matches the output of the previous command. This is an example where the interface ID is 07:00.0
</div>

    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"
    
<img src="./labs_files/lab9/figs/pktgen_command.png" width="650"><br>

In the figure above, the command ```pktgen``` is used to run the packet generator considering the following arguments:

•	```-l```: List of cores to run on.<br>
•	```-n```: Number of memory channels.<br>
•	```-a```: The ID of allowed interfaces (the command shows an example where interface 07:00.0 is used).<br>
•	```-P```: Enable promiscuous mode on all ports.<br>
•	```-m```: Matrix for mapping ports to logical cores.<be>

In this step, we are invoking pktgen, using cores 2 cores (0 and 1) as specified in the ```-l``` parameter with 4 memory channels as specified in the ```-n``` parameter. The interface allowed ```-a``` has the ID 07:00.0 in this example. We enabled promiscuous mode ```-P``` and mapped CPU core 1 to handle the rx and tx ports of port 0 as specified in the ```-m``` parameter.

As soon a pktgen starts the main screen is displayed:

<img src="./labs_files/lab9/figs/pktgen_main.png" width="650"><br>

## Step 9.3: Configuring pktgen

In pktgen, we will set the source and destination MAC and IP addresses along with the desired packet size of the generated packets. RSS will distribute packets that belong to the same flow on each of the four different pipelines running on four different cores. Therefore, we will have to generate the packets from a range of flows. To do so, we will generate packets from a range of source and destination IP addresses. 

In pktgen terminal, enter the following command to navigate the page displaying the settings of packets sent from a range of flows:

    page range

<img src="./labs_files/lab9/figs/pktgen_range.png" width="650"><be>

<hr>

Enter the following commands to set the source MAC address:

    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01

<img src="./labs_files/lab9/figs/pktgen_srcmac.png" width="650"><be>

To modify the settings in the rage page we used the ```range``` command. To set the source MAC address of the generated packets sent from port ID 0, we used the ```src mac``` command and since the source MAC addresses are not randomized, we will set the starting value ```start```, minimum value ```min``` and maximum value ```max``` to be ```00:00:00:00:00:01``` which is the MAC address of server1. 

<hr>

Enter the following commands to set the destination MAC address:

    range 0 dst mac start 00:00:00:00:00:02
    range 0 dst mac min 00:00:00:00:00:02
    range 0 dst mac max 00:00:00:00:00:02

<img src="./labs_files/lab9/figs/pktgen_dstmac.png" width="650"><be>

To modify the settings in the rage page we used the ```range``` command. To set the destination MAC address of the generated packets sent from port ID 0, we used the ```dst mac``` command and since the destination MAC addresses are not randomized, we will set the starting value ```start```, minimum value ```min``` and maximum value ```max``` to be ```00:00:00:00:00:02``` which is the MAC address of server2. 

<hr>

Enter the following commands to set the source IP address:

    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1

<img src="./labs_files/lab9/figs/pktgen_srcip.png" width="650"><be>

To modify the settings in the rage page we used the ```range``` command. To set the source IP address of the generated packets sent from port ID 0, we used the ```src ip``` command. The starting value ```start``` and minimum value ```min``` are both set as the IP ```1.1.1.1```. The rage will reach a maximum value ```max``` to be ```250.250.250.250``` by incrementing each octet by ```0.0.0.1``` and specified in the ```inc```. 

<hr>

Enter the following commands to set the destination IP address:

    range 0 dst ip start 1.1.1.1
    range 0 dst ip min 1.1.1.1
    range 0 dst ip max 250.250.250.250
    range 0 dst ip inc 0.0.0.1

<img src="./labs_files/lab9/figs/pktgen_dstip.png" width="650"><be>

To modify the settings in the rage page we used the ```range``` command. To set the destination IP address of the generated packets sent from port ID 0, we used the ```dst ip``` command. The starting value ```start``` and minimum value ```min``` are both set as the IP ```1.1.1.1```. The rage will reach a maximum value ```max``` to be ```250.250.250.250``` by incrementing each octet by ```0.0.0.1``` and specified in the ```inc```. 

<hr>

Enter the following commands to set the packet size:

    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500

<img src="./labs_files/lab9/figs/pktgen_size.png" width="650"><be>

To modify the settings in the rage page we used the ```range``` command. To set the size in bytes of the generated packets sent from port ID 0, we used the ```size``` command and since the packet size is not randomized, we will set the starting value ```start```, minimum value ```min``` and maximum value ```max``` to be ```1500``` to set the packet size to 1500 bytes. 

<hr>

Enter the following command to enable generating packets from a range of flows and going back to the main page:

    enable 0 range
    page main

<img src="./labs_files/lab9/figs/pktgen_enable.png" width="650"><be>

## Step 9.4: Start sending packets

In pktgen terminal execute the following commands to start generating and sending packets from server 1 to server 2:

    start 0

<img src="./labs_files/lab9/figs/pktgen_start.png" width="600"><br>

To start sending packets, we used the ```start``` command followed by the port list. In this case, we are using one port with ID 0. We can observe the rate at which the packets are sent from server 1 and received from server 2. In the grey box, the first two lines display the number of packets sent and received per second, and the last line displays the sending and receiving throughput in Mbyte per second. The result in the screenshot above shows that pktgen is generating packets at a rate close to 100 Gbyte per second. These packets are sent to server 2 on which the DPDK pipeline is running, and they are reflected back to server 1 at a similar rate of around 100 Gbytes per second. 

## Step 9.5: Stop sending packets

In pktgen terminal execute the following commands to stop generating and sending packets from server 1 to server 2:

    stop 0

<img src="./labs_files/lab9/figs/pktgen_stop.png" width="600"><be>

To stop sending packets, we used the ```stop``` command followed by the port list. In the grey box, we can see that the throughput dropped to 0, indicating that packets are not sent or received.

## Step 9.6: Stop the running pipeline

Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab9/figs/pipeline_stop.png" width="650"><br>

# Step 10: Delete the slice

This concludes Lab 9. Please delete your slice when you are done with your experiment.

In [49]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab9")
slice.delete()

# References

1.	H. Zhu, “Data Plane Development Kit (DPDK): A Software Optimization Guide to the User Space-based Network Application”, CRC Press, 2020.
2.	NVIDIA, “What is CPU Affinity?”, [Online]. Available: https://tinyurl.com/mwb57c5s.
3.	DPDK, “Data Plane Development Kit documentation”, Release 2.2.0, 2016.
4.	DPDK, “rte_pipeline.h File Reference”, [Online]. Available: https://tinyurl.com/sh9254cs.
5.  pktgen, “Pktgen-DPDK”, [Online]. Available: https://github.com/pktgen/Pktgen-DPDK